# Análise de dados do Airbnb

## Por Eugênio, Isabella, João, Julia e Rafael

* city: cidade da listagem (nossa variável)
* weekdays: retorna True se é dia da semana (e não fim de semana) (nossa variável)
* realSum: o preço de acomodação para duas pessoas por duas noites, em euros
* room_type: tipo de acomodação (quarto privado, quarto compartilhado ou casa inteira)
* room_shared: retorna True se o quarto for compartilhado
* room_private: retorna True se o quarto for privado
* person_capacity: número máximo de pessoas 
* host_is_superhost: retorna True se o anfitrião é um Superhost
* multi: retorna True se são oferecidos 2 a 4 quartos diferentes
* biz: retorna True se são oferecidos mais de 4 quartos diferentes
* cleanliness_rating: avaliação da limpeza
* guest_satisfaction_overall: nota geral da listagem
* bedrooms: número de quartos (0 para estúdios)
* dist: distância até o centro da cidade em km
* metro_dist: distância do metrô mais próximo em km
* attr_index: índice de atratividade da localização da listagem
* attr_index_norm: índice de atração normalizado (0-100)
* rest_index: índice de restaurantes da localização da listagem
* rest_index_norm: índice de restaurantes normalizado (0-100)
* lng: longitude da localização da listagem
* lat: latitude da localização da listagem

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns
%matplotlib inline
import glob
import os
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# União de todos os arquivos (20 arquivos, dois para cada cidade - em dias da semana e fim de semana)
# Criação de colunas para as cidades e para definir se é um dia da semana ou não
all_files = glob.glob('*.csv')

li = []

for filename in all_files:
    print(filename)
    df_temp = pd.read_csv(filename, index_col=0)
    df_temp["city"] = filename[:filename.index('_')].title()
    df_temp["weekdays"] = True if filename[filename.index('_')+1:filename.index('.')] == "weekdays" else False
    li.append(df_temp)

df = pd.concat(li, axis=0, ignore_index=True)
display(df)

In [ ]:
# Reordenação das colunas do dataframe
df = df[['city', 'weekdays', 'realSum', 'room_type', 'room_shared', 'room_private',
       'person_capacity', 'host_is_superhost', 'multi', 'biz',
       'cleanliness_rating', 'guest_satisfaction_overall', 'bedrooms', 'dist',
       'metro_dist', 'attr_index', 'attr_index_norm', 'rest_index',
       'rest_index_norm', 'lng', 'lat']]

## Visualização básica do Dataframe

In [ ]:
# Verifica-se que o dataframe possui 21 colunas, das quais 4 são booleanas, 13 são numéricas, 2 são categóricas (texto)
# e 2 são numéricas contendo informações de latitude e longitude
df.info()

In [ ]:
# Pegando 5 linhas aleatórias para verificar como os dados estão dispostos
df.sample(5).T

In [ ]:
# Gerando descrições estatísticas básicas
df.describe(include="all").T

In [ ]:
# Verificando número de linhas e colunas do DF
df.shape

In [ ]:
# Como funciona a correlação entre as variáveis?
# Verifica-se que a correlação é baixa, com exceção de 
plt.figure(figsize=(16, 6))
sns.heatmap(df[df['realSum']<2000].corr(), cmap="coolwarm", vmin=-1, vmax=1, annot=True)

## Análise qualitativa dos imóveis

In [ ]:
# Como estão distribuídos nas cidades as informações de locação?
# Verifica-se que dos dados apresentados, quase 50% dos imóveis
# locados pela Airbnb estão em Londres, Roma e Paris
df1 = df.groupby('city').weekdays.count().reset_index()
df1 = df1.rename(columns={'weekdays': 'count'})
df1 = df1.sort_values('count', ascending=True)
df1['percent'] = round(100*df1['count']/df1['count'].sum(),1)
px.bar(data_frame=df1, y='city', x='count', text='percent', title='Imóveis por Cidade: Quase 50% dos imóveis locados ficam em Londres, Roma e Paris',
       labels={'city': 'Cidade', 'count':'Quantidade', 'percent':'Porcentagem'},
       hover_data=['count', 'percent'], hover_name='city', width=1000)

In [ ]:
# E como é a distribuição dos diferentes tipos de imóveis?
# - Entire home/apt = imóvel completo
# - Private room = quarto privativo dentro de um imóvel/hotel
# - Shared room = quarto compartilhado
# Verifica-se que 63% dos registros de locação são para imóveis completos.
df2 = df.groupby('room_type').weekdays.count().reset_index()
df2 = df2.rename(columns={'weekdays': 'count'})
df2 = df2.sort_values('count', ascending=True)
df2['percent'] = round(100*df2['count']/df2['count'].sum(),1)

px.bar(data_frame=df2, y='room_type', x='count', text='percent', title='Tipos de Imóvel: 63% das locações são para o imóvel completo ("Entire home/apt")', 
       labels={'room_type': 'Tipo de Imóvel', 'count':'Quantidade', 'percent':'Porcentagem'},
       hover_data=['room_type', 'count', 'percent'], width=1000)

In [ ]:
# A distribuição fica diferente por cidade?
# Aparentemente sim, verifica-se que ao agrupar por cidade, em algumas cidades
# há mais locação de quartos privados que imóveis completos, como é o caso de Londres,
# Barcelona e Berlim.
df3 = df.groupby(['city', 'room_type']).weekdays.count().reset_index()
df3 = df3.rename(columns={'weekdays': 'count'})

px.bar(data_frame=df3, x='room_type', y='count', facet_col='city', title='Tipos de Imóvel por Cidade: Em Barcelona, Berlim e Londres temos mais imóveis do tipo "Private Room"', 
       labels={'room_type': 'Tipo', 'count':'Quantidade'}, width=1200)

In [ ]:
px.bar(data_frame=df3, x='city', y='count', facet_col='room_type', text='count', title='Tipos de Imóvel por Cidade: Apesar de ficar em 5º no ranking de quantidade de imóveis, Atenas fica em 3º na maior quantidade de "Entire home/apt"', 
       labels={'room_type': 'Tipo de Imóvel', 'count':'Quantidade'}, width=1400)

In [ ]:
# Como se comporta a distribuição da capacidade de pessoas das locações?
# Verifica-se que imóveis para 2 a 4 pessoas são mais comuns
df4 = df.groupby('person_capacity').weekdays.count().reset_index()
df4 = df4.rename(columns={'weekdays': 'count'})

sns.barplot(df4, x = 'person_capacity', y='count').set(title = 'Os imóveis mais comuns são para 2 a 4 pessoas',
xlabel='Capacidade de pessoas', ylabel='Quantidade');

In [ ]:
# Como funciona a distribuição da capacidade de pessoas por diferentes tipos de imóveis?
# Verifica-se, como esperado, que imóveis completos podem possuir maior capacidade,
# enquanto quartos privativos geralmente possuem capacidade para até 4 pessoas.

df5 = df.groupby(['room_type','person_capacity']).weekdays.count().reset_index()
df5 = df5.rename(columns={'weekdays': 'count'})

sns.barplot(df5, x = 'room_type', y='count', hue="person_capacity").set(title = 'O "Entire home/apt" possui mais variedades de capacidade de pessoas',
                                                                           xlabel='Tipo de imóvel', ylabel='Quantidade');

In [ ]:
# E como estão distribuídas a capacidade de pessoas por cidade?
# Verifica-se que com exceção de Atenas e Budapeste a maioria das locações tem
# capacidade de 2 pessoas.
# Como esperado verifica-se relação da capacidade com o tipo de imóvel. Em Barcelona,
# Londres e Berlim, onde haviam mais imóveis do tipo quarto privado, a maioria da capacidade
# está em 2 pessoas.
df6 = df.groupby(['city','person_capacity']).weekdays.count().reset_index()
df6 = df6.rename(columns={'weekdays': 'count'})

plt.figure(figsize=(10,8))
sns.barplot(df6, y = 'city', x='count', hue="person_capacity").set(title = 'Com exceção de Atenas e Budapeste, todas as cidades costumam oferecer mais imóveis com capacidade para 2 pessoas',
                                                                           xlabel='Quantidade', ylabel='Cidade');

In [ ]:
# Como ficam as proporções de superhosts por cidade?
# Verifica-se que a maior proporção de superhosts é em Atenas
df7 = df.groupby(['city', 'host_is_superhost']).weekdays.count().reset_index()
df7 = df7.rename(columns={'weekdays': 'count'})

sns.barplot(df7, y='city', x='count', hue="host_is_superhost").set(title='Atenas tem maior proporção de superhosts', xlabel='Quantidade', ylabel='Cidade');

In [ ]:
# Verificou-se que havia alta correlação entre a nota de limpeza
# e a nota geral
df.plot(x="cleanliness_rating", y="guest_satisfaction_overall", kind="scatter", title="Quanto maior a nota de limpeza, maior a satisfação")

In [ ]:
sns.boxplot(df, x="guest_satisfaction_overall").set(title="Clientes em geral estão satisfeitos")

In [ ]:
# Como estão as notas de limpeza?
sns.boxplot(df, x="cleanliness_rating").set(title="Nota 10 em limpeza para a maioria das locações")

In [ ]:
# Como se relacionam o preço, o status de superhost e a satisfação dos hóspedes?
# Grafico barra horizontal separado por cidade
sns.barplot(df, y="city", x="guest_satisfaction_overall", hue="host_is_superhost", hue_order=[True, False])
plt.legend(title='Superhost')
plt.xlabel('Satisfaction')
plt.ylabel('City')
plt.title('Superhost são melhores avaliados')
#Resposta - Sim, Superhosts são mais bem avaliados

In [ ]:
# Há diferença de avaliação para diferentes tipos de locatários?
# biz = false & multi = false: quem só tem um anúncio
# biz = false & multi = true: quem tem de 2 a 4 anúncios
# biz = true & multi = false = quem tem >4 anúncios
def checa_tipo(row):
    if row['biz'] == False and row['multi'] == False:
        valor='1'
    elif row['biz'] == False and row['multi'] == True:
        valor='2-4'
    elif row['biz'] == True and row['multi'] == False:
        valor='>4'
    return pd.Series(valor)
df['cat'] = df.apply(checa_tipo, axis=1)

In [ ]:
sns.barplot(df, x="cat", y="guest_satisfaction_overall", hue='host_is_superhost', hue_order=[True, False], order=['1', '2-4', '>4'])
plt.xlabel('Quantidade de anúncios do anfitrião')
plt.ylabel('Satisfação do cliente')
plt.title('Anfitriões que não são superhost e tem menos anúncios satisfazem mais seus clientes')
plt.legend(title='Superhost')

## Análise de preços

In [ ]:
# Como é a média de valores de locação nas cidades analisadas?
sns.barplot(df, x="realSum", y="city").set(title='Amsterdam aparenta ter a média de preços maior')

In [ ]:
# Como é o boxplot dos valores de locação?
# Verifica-se que há muitos outliers com valores excessivamente alto
sns.boxplot(df["realSum"]).set(title='Muitos outliers!')

In [ ]:
sns.boxplot(df, x="realSum", y="city").set(title='Muitos outliers inclusive por cidade')

In [ ]:
# Foram tantos outliers que para visualizar o boxplot de forma mais limpa,
# foi necessário filtrar o valor de locação para valores abaixo de EUR 1.200
# por 2 noites por casal
# Verifica-se que Atenas tem a mediana mais baixa.
sns.boxplot(df[df["realSum"]<1200], x="realSum", y="city").set(title='Atenas tem menor mediana de preços')

In [ ]:
# Há diferença entre valor para semana ou final de semana?
# Verifica-se que Atenas e Paris possuem maior valor médio de
# locação para dias de semana que para finais de semana.
sns.barplot(df, x="realSum", y="city", hue="weekdays", hue_order=[True, False])

In [ ]:
# Criando cópia do Dataframe e limitando o valor de locação em 1.500
copia_df = df.copy()
copia_df = copia_df[copia_df["realSum"]<1500]
# O valor apontado no DF é por 2 noites para 2 pessoas, vamos então
# calcular o valor por pessoa por noite:
copia_df["price_per_person"] = copia_df["realSum"] / 4
copia_df.describe(include="all", percentiles=[0,0.25,0.5,0.75,0.95,1]).T

In [ ]:
# Uma casa para mais pessoas (maior person_capacity) é necessariamente mais cara?
sns.barplot(df, x="realSum", y="city", hue="person_capacity").set(title='Sim, maior capacidade = maior preço')

In [ ]:
# O preço varia com a distância ao centro da cidade?
sns.lmplot(data=copia_df, x="dist", y="realSum", logx=True, hue="city", height=5, aspect=2, scatter=False).set(title='Imóveis mais perto do centro são mais caros', xlabel='Distance from Center', ylabel='Price')

In [ ]:
# O preço varia com a distância a estação de metrô?
sns.lmplot(data=copia_df, x="metro_dist", y="realSum", logx=True, hue="city", height=5, aspect=2, scatter=False).set(title='Imóveis mais perto do metrô são mais caros', xlabel='Distance from metro', ylabel='Price')

In [ ]:
# O preço varia com o índice de atratividade?
sns.lmplot(data=copia_df, x="attr_index_norm", y="realSum", logx=True, hue="city", height=5, aspect=2, scatter=False).set(title='Imóveis com maior índice de atratividade são mais caros', xlabel='Índice de atratividade', ylabel='Price')

In [ ]:
# O preço varia com a proximidade de restaurantes atrativos?
sns.lmplot(data=copia_df, x="rest_index_norm", y="realSum", logx=True, hue="city", height=5, aspect=2, scatter=False).set(title='Imóveis com maior proximidade de restaurantes atrativos são mais caros', xlabel='Índice de atratividade de restaurante', ylabel='Price')

## Visualização em mapas

In [ ]:
# Visualizando os preços em locais através de mapas
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat))
geo_copia_df = gpd.GeoDataFrame(copia_df, geometry=gpd.points_from_xy(copia_df.lng, copia_df.lat))

#scatter plot com geo_df2
#use strong colors for the points
#change collor from light to dark as the price increases
#cor = preto
fig = px.scatter_mapbox(geo_copia_df, lat="lat", lon="lng", size="realSum", hover_name="city", zoom=3, height=500, color="realSum", color_continuous_scale=px.colors.sequential.Blackbody)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(mapbox_style="open-street-map")
fig.show()


In [ ]:
fig = px.scatter_mapbox(geo_copia_df, lat="lat", lon="lng", hover_name="city", hover_data=["weekdays"], size="price_per_person",color = 'price_per_person', zoom=4, height=500, color_continuous_scale=px.colors.sequential.Viridis_r, range_color=[20,150])
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://a.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()